In [3]:
import pandas as pd

In [2]:
movies = pd.read_csv('../raw_dataset/tmdb_5000_movies.csv')
credit = pd.read_csv('../raw_dataset/tmdb_5000_credits.csv') 

In [3]:
movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [4]:
credit.head()

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [5]:
#Renaming Column id to movie_id
movies.rename(columns= {'id':'movie_id'}, inplace=True)

In [6]:
movies = movies.merge(credit, on='movie_id')

In [7]:
movies.iloc[0]


budget                                                          237000000
genres                  [{"id": 28, "name": "Action"}, {"id": 12, "nam...
homepage                                      http://www.avatarmovie.com/
movie_id                                                            19995
keywords                [{"id": 1463, "name": "culture clash"}, {"id":...
original_language                                                      en
original_title                                                     Avatar
overview                In the 22nd century, a paraplegic Marine is di...
popularity                                                     150.437577
production_companies    [{"name": "Ingenious Film Partners", "id": 289...
production_countries    [{"iso_3166_1": "US", "name": "United States o...
release_date                                                   2009-12-10
revenue                                                        2787965087
runtime                               

### Important Features to be Extracted (Separated)

TEXT - IMPORTANT - original_title genres, movie_id, keywords, original_language, original_title, production_companies, crew, cast <br>
TEXT - LESS - overview, tagline <br>


- *Numrical data <font color=red>__NOT__</font> taken into consideration in this particular project <br>
- *will deal with later <br>
NUM - IMPORTANT - release_date, popularity, vote_average <br>
NUM - LESS <br>

***

### Text And Num Data Separation

In [10]:
movies_text_Data = movies[['movie_id','original_title','genres','keywords','original_language', 'production_companies', 'crew', 'cast']]
movies_num_Data = movies[['release_date', 'popularity', 'vote_average']] # Not using currently

In [15]:
# This files need to be updated whenever new database 
movies_text_Data.to_csv("../feature_extracted_dataset/tmdb_text_data.csv", index=False)

***

### Cleaning and Processing 
- to be fed to text-vectorization model

In [4]:
movies_text_Data = pd.read_csv("../feature_extracted_dataset/tmdb_text_data.csv")
movies_text_Data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   movie_id              4803 non-null   int64 
 1   original_title        4803 non-null   object
 2   genres                4803 non-null   object
 3   keywords              4803 non-null   object
 4   original_language     4803 non-null   object
 5   production_companies  4803 non-null   object
 6   crew                  4803 non-null   object
 7   cast                  4803 non-null   object
dtypes: int64(1), object(7)
memory usage: 300.3+ KB


#### Handling one trade-off

In [5]:
print(f"English: {len(movies_text_Data[movies_text_Data['original_language']=='en'])}")
print(f"NOT English: {len(movies_text_Data[movies_text_Data['original_language']!='en'])}")

English: 4505
NOT English: 298


In [4]:
movies_text_Data.iloc[786]

movie_id                                                           381902
original_title                                               西游记之孙悟空三打白骨精
genres                  [{"id": 28, "name": "Action"}, {"id": 12, "nam...
keywords                          [{"id": 207411, "name": "monkey king"}]
original_language                                                      zh
production_companies            [{"name": "Filmko Pictures", "id": 9175}]
crew                    [{"credit_id": "56bdb3e5925141734a002d66", "de...
cast                    [{"cast_id": 0, "character": "The Monkey King"...
Name: 786, dtype: object

Removing Movies other than en (English) from original_language feature <br>
This will remove other language titles from 'original_title' like '西游记之孙悟空三打白骨' which is at index 786 <br>
So that later on when we add 'original_title' to the 'tags' it will not cause any error <br>

This will increase the performance when searchin for similar titles like :
- Star Wars Collection
- Harry Potter Collection
- Lord Of The Rings Collection
<br>

__But__ we have to remove the data 298 movies (~6% of Dataset) from the data-set (Ouch...😅) <br>

This is a <font color='pink'>__Trade Off__</font><br>

__HERE__ I am chosing not to remove these movies as 298 is not a small number <br>

__HOWEVER__ if you want to delete uncomment the below block



In [5]:
# x = movies_text_Data['original_language'] != 'en'
# del_index = x[x].index
# del_index
# movies_text_Data.drop(index=del_index, inplace=True)

***

__Continuing__.....

In [6]:
import ast

In [7]:
def convert(text:str): #genres, keywords, production_companies
    L = []
    for i in ast.literal_eval(text):
        L.append(i['name']) 
    return L

In [8]:
movies_text_Data.head(1)

,movie_id,original_title,genres,keywords,original_language,production_companies,crew,cast
0,19995,Avatar,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""..."


In [9]:
movies_text_Data['genres'] = movies_text_Data['genres'].apply(convert)
movies_text_Data['keywords'] = movies_text_Data['keywords'].apply(convert)
movies_text_Data['production_companies'] = movies_text_Data['production_companies'].apply(convert)

movies_text_Data.head(1)

,movie_id,original_title,genres,keywords,original_language,production_companies,crew,cast
0,19995,Avatar,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...",en,"[Ingenious Film Partners, Twentieth Century Fo...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""..."


In [10]:
def convert_cast(text:str): #cast
    L = []
    count = 0
    for i in ast.literal_eval(text):
        if count < 5:
            L.append(i['name'])
        count+=1
    return L 

In [11]:
movies_text_Data['cast'] = movies_text_Data['cast'].apply(convert_cast)

In [12]:
movies_text_Data.iloc[0].cast

['Sam Worthington',
 'Zoe Saldana',
 'Sigourney Weaver',
 'Stephen Lang',
 'Michelle Rodriguez']

In [13]:
def fetch_director(text): #director from 'crew'
    L = []
    for i in ast.literal_eval(text):
        if i['job'] == 'Director':
            L.append(i['name'])
    return L

In [14]:
movies_text_Data['crew'] = movies_text_Data['crew'].apply(fetch_director)

In [15]:
movies_text_Data.head(1)

,movie_id,original_title,genres,keywords,original_language,production_companies,crew,cast
0,19995,Avatar,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...",en,"[Ingenious Film Partners, Twentieth Century Fo...",[James Cameron],"[Sam Worthington, Zoe Saldana, Sigourney Weave..."


In [16]:
def collapse_and_lower(L): # removing all spaces in between and lowering all uppercase letters
    L1 = []
    L_new = []
    
    for i in L:
        L1.append(i.replace(" ",""))
    
    for i in L1:
        L_new.append(i.lower())

    return L_new

In [17]:
# Renaming the 'crew' column to 'director'
movies_text_Data.rename(columns= {'crew':'director'}, inplace= True) 

In [18]:
movies_text_Data['director'] = movies_text_Data['director'].apply(collapse_and_lower)
movies_text_Data['cast'] = movies_text_Data['cast'].apply(collapse_and_lower)
movies_text_Data['production_companies'] = movies_text_Data['production_companies'].apply(collapse_and_lower)
movies_text_Data['keywords'] = movies_text_Data['keywords'].apply(collapse_and_lower)
movies_text_Data['genres'] = movies_text_Data['genres'].apply(collapse_and_lower)

In [19]:
movies_text_Data.head(1)

,movie_id,original_title,genres,keywords,original_language,production_companies,director,cast
0,19995,Avatar,"[action, adventure, fantasy, sciencefiction]","[cultureclash, future, spacewar, spacecolony, ...",en,"[ingeniousfilmpartners, twentiethcenturyfoxfil...",[jamescameron],"[samworthington, zoesaldana, sigourneyweaver, ..."


In [20]:
# Converting the original_language column to list of strings
def convert_org_lang(text:str):
    l = []
    l.append(text)
    return l

In [21]:
movies_text_Data['original_language'] = movies_text_Data['original_language'].apply(convert_org_lang)

In [22]:
movies_text_Data.iloc[0].original_language

['en']

<font color=red>__IMPORTANT__</font>
- adding 'original_language' to 'tags' might cause some bias as:<br>
English: 4505 <br>
NOT English: 298
- there are __a lot__ of english movies
- Take this into consideration if you want to add 'original_language'
- here, i have added them.

In [28]:
def title_to_list(obj):
    l = []
    tmp = obj.lower()
    l.append(tmp.replace(" ",""))
    return l

In [29]:
movies_text_Data['title_alt'] = movies_text_Data['original_title'].apply(title_to_list)

In [30]:
movies_text_Data['tags'] =  movies_text_Data['original_language']+ movies_text_Data['title_alt'] + movies_text_Data['genres'] + movies_text_Data['keywords'] + movies_text_Data['director'] + movies_text_Data['cast'] + movies_text_Data['production_companies']

In [31]:
movies_text_Data.head(1)

,movie_id,original_title,genres,keywords,original_language,production_companies,director,cast,title_alt,tags
0,19995,Avatar,"[action, adventure, fantasy, sciencefiction]","[cultureclash, future, spacewar, spacecolony, ...",[en],"[ingeniousfilmpartners, twentiethcenturyfoxfil...",[jamescameron],"[samworthington, zoesaldana, sigourneyweaver, ...",[avatar],"[en, avatar, action, adventure, fantasy, scien..."


In [32]:
final_movies = movies_text_Data.drop(columns=['genres', 'keywords','original_language', 'production_companies', 'director','cast','title_alt']) 

In [33]:
final_movies.iloc[0]

movie_id                                                      19995
original_title                                               Avatar
tags              [en, avatar, action, adventure, fantasy, scien...
Name: 0, dtype: object

In [34]:
final_movies.head()

,movie_id,original_title,tags
0,19995,Avatar,"[en, avatar, action, adventure, fantasy, scien..."
1,285,Pirates of the Caribbean: At World's End,"[en, piratesofthecaribbean:atworld'send, adven..."
2,206647,Spectre,"[en, spectre, action, adventure, crime, spy, b..."
3,49026,The Dark Knight Rises,"[en, thedarkknightrises, action, crime, drama,..."
4,49529,John Carter,"[en, johncarter, action, adventure, sciencefic..."


In [38]:
# Saving the Cleaned and processed data
final_movies.to_csv("../cleaned_and_processed_dataset/cleaned_data_to_be_vectorized.csv", index = False)

In [40]:
df = pd.read_csv("../cleaned_and_processed_dataset/cleaned_data_to_be_vectorized.csv")
df.head()

,movie_id,original_title,tags
0,19995,Avatar,"['en', 'avatar', 'action', 'adventure', 'fanta..."
1,285,Pirates of the Caribbean: At World's End,"['en', ""piratesofthecaribbean:atworld'send"", '..."
2,206647,Spectre,"['en', 'spectre', 'action', 'adventure', 'crim..."
3,49026,The Dark Knight Rises,"['en', 'thedarkknightrises', 'action', 'crime'..."
4,49529,John Carter,"['en', 'johncarter', 'action', 'adventure', 's..."


In [41]:
df.iloc[0].tags

"['en', 'avatar', 'action', 'adventure', 'fantasy', 'sciencefiction', 'cultureclash', 'future', 'spacewar', 'spacecolony', 'society', 'spacetravel', 'futuristic', 'romance', 'space', 'alien', 'tribe', 'alienplanet', 'cgi', 'marine', 'soldier', 'battle', 'loveaffair', 'antiwar', 'powerrelations', 'mindandsoul', '3d', 'jamescameron', 'samworthington', 'zoesaldana', 'sigourneyweaver', 'stephenlang', 'michellerodriguez', 'ingeniousfilmpartners', 'twentiethcenturyfoxfilmcorporation', 'duneentertainment', 'lightstormentertainment']"